In [38]:
import torch
from PIL import Image
from torchvision import transforms
import numpy as np

In [ ]:
model = torch.hub.load('pytorch/vision:v0.10.0', 'resnet18', pretrained=True)
if torch.cuda.is_available():
    print("CUDA is available! Running on gpu")
    model = model.to('cuda')
model.eval()

In [ ]:
inputPath = "Scripts/cat3/cat.mp4"
frames_number = 14

# Load the images
images = {}
for i in range(1,frames_number + 1):
    img_name = inputPath + "/" + str(i) + ".jpg" 
    img = Image.open(img_name)
    images[img_name] = (img)

images

In [ ]:
activation = None

In [ ]:
def hook(model, input, output):
    global activation
    activation = output

In [25]:
preprocess = transforms.Compose([
    transforms.Resize(512),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])
latent = {}
for i in images:
    input_tensor = preprocess(images[i])   
    input_batch = input_tensor.unsqueeze(0)

    # move the input and model to GPU for speed if available
    if torch.cuda.is_available():
        input_batch = input_batch.to('cuda')

    model.layer3[1].conv2.register_forward_hook(hook)

    with torch.no_grad():
        model(input_batch)

    latent[i] = activation[0]

In [26]:
latent['Scripts/cat3/cat.mp4/1.jpg'].shape

torch.Size([512, 16, 29])

In [68]:
from sklearn_extra.cluster import KMedoids

#  initial shape 512 x 16 x 29, need to reshape everything to 1 x (512*16*29) then stack to 
flattened_arrays = [arr.flatten() for arr in latent.values()]


flattened_data = np.vstack(flattened_arrays)

med_model = KMedoids(n_clusters=1, random_state=0)

med_model.fit(flattened_data)

# Trovare il medoide
medoid_index = med_model.medoid_indices_[0]

medoid_index

10